## RadioCut integrado con IA

### A - Instanciación de la AI Mistral

In [1]:
import os
from dotenv import load_dotenv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

model = "mistral-large-latest"

client = MistralClient(api_key=api_key)


##### Prueba del chat

In [2]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content="Cuál es el significado de 'ubicuo'?")]
)

print(chat_response.choices[0].message.content)

La palabra "ubicuo" se utiliza para describir algo que está presente en todas partes o en muchos lugares al mismo tiempo. Proviene del latín "ubique", que significa "en todas partes". En un contexto más amplio, puede referirse a algo que es omnipresente o que se encuentra en múltiples ubicaciones simultáneamente. Por ejemplo, se puede decir que el internet es ubicuo porque está disponible en muchos lugares y es accesible desde diversas plataformas y dispositivos.


### B - Pasos para acceder a los recortes

#### 1ero: consultar "robots.txt" para ajustar crawl-delay

In [3]:
import requests

def get_robots_txt(url):
    if not url.endswith('/'):
        url += '/'
    
    robots_url = url + "robots.txt"
    
    response = requests.get(robots_url)
    
    if response.status_code == 200:
        return response.text
    else:
        return None

website_url = "https://radiocut.fm"
robots_txt = get_robots_txt(website_url)

if robots_txt:
    print("robots.txt content:")
    print(robots_txt)
else:
    print("robots.txt no encontrado")

robots.txt content:
User-agent: *
Crawl-delay: 30
Request-rate: 1/3
Sitemap: https://radiocut.fm/sitemap.xml
Disallow: /*/delete$



#### 2do: solicitar los recortes del día

In [4]:
import requests
import time

def fetch_url(url):
    response = requests.get(url)
    return response

# Recortes del día
urls = ["https://ar.radiocut.fm/search/?type=cut&page=4&created=1",
 "https://ar.radiocut.fm/search/?type=cut&page=4&created=2"]

crawl_delay = 30  # segundos
request_rate = 1 / 3  # 1 solicitud cada 3 segundos

docs = []
for url in urls:
    response = fetch_url(url)
    if response.status_code == 200:
        print("Fetched:", url)
        docs.append(response.text)

    else:
        print("Failed to fetch:", url)
    
    # Esperar de acuerdo al tiempo del crawl (rebaje)
    time.sleep(crawl_delay)
    

Fetched: https://ar.radiocut.fm/search/?type=cut&page=4&created=1
Fetched: https://ar.radiocut.fm/search/?type=cut&page=4&created=2


In [5]:
len(docs)

2

In [6]:
docs

['\n<!DOCTYPE html>\n<html lang="es">\n<head>\n<link rel="preconnect dns-prefetch" href="https://www.googletagmanager.com" />\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width,initial-scale=1.0" />\n<meta name="alexaVerifyID" content="VnZ_Et-Jpg6HoBQRuWYP1dYWaa0" />\n<meta property="fb:app_id" content="471603786214644" />\n<meta property="og:site_name" content="RadioCut" />\n<meta property="og:image" content="https://static.radiocut.com.ar/images/radio_cut_96.png" />\n<meta name="description" content="\n            \n            Escucha radios online de Argentina\n            \n          y programas de la radio FM, AM, Noticias, Música... Retrocede horas y días atrás y crea tus propios recortes de la radio.\n        ">\n<meta name="keywords" content="radios,radiodifusion,programa de radio,noticias,archivo de radios, podcast, deportes,musica,radiocut">\n<meta name="apple-itunes-app" content="app-id=1120326976">\n<meta name="google-play-app" content="app-id=com.l

#### 3ero: scrapping o "recauchute"

In [7]:
from bs4 import BeautifulSoup

## Librerías FECHA
import re
from datetime import datetime, timedelta
import pytz
import locale

# Establece la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

soup = BeautifulSoup(docs[0], 'html.parser')

# Métodos FECHA
def clean_duration(duration_text):
    return duration_text.replace("Dur: ", "").strip()

def convert_duration_to_date(duration_text):
    pattern = re.compile(r'(\d+)\s*año|(\d+)\s*mes|(\d+)\s*semana|(\d+)\s*día|(\d+)\s*hora|(\d+)\s*minuto')
    matches = pattern.findall(duration_text)
    
    years = months = weeks = days = hours = minutes = 0
    for match in matches:
        if match[0]: years = int(match[0])
        if match[1]: months = int(match[1])
        if match[2]: weeks = int(match[2])
        if match[3]: days = int(match[3])
        if match[4]: hours = int(match[4])
        if match[5]: minutes = int(match[5])
    
    now = datetime.now(pytz.timezone('America/Argentina/Buenos_Aires'))
    time_delta = timedelta(days=(years * 365 + months * 30 + weeks * 7 + days), hours=hours, minutes=minutes)
    result_date = now - time_delta
    utc_dt = result_date.astimezone(pytz.utc)
    
    # Formatea el día y el mes manualmente para evitar ceros a la izquierda
    day = utc_dt.day
    month = utc_dt.strftime('%B')
    year = utc_dt.year
    time_str = utc_dt.strftime('%H:%M:%S')
    readable_date = f'{day} de {month} de {year} {time_str}'
    
    return readable_date

data_list = []

# Cucharada de sopa
containers = soup.find_all('div', class_='col-sm-10 col-xs-8')

for container in containers:
    # Extraer el título
    title_tag = container.find('a')
    title = title_tag.text.strip() if title_tag else None

    if title:  # Sólo sigue si hay título
        # Extrae la duración y la emprolija
        duration_tag = container.find('span', class_='text-muted')
        duration = clean_duration(duration_tag.text) if duration_tag else None

        # Extrae la descripción y la emprolija
        description_tag = container.find('p', class_='col-sm-12 description text-left')
        description = description_tag.text.strip() if description_tag else None

        # Extrae la fecha de creación del contenedor actual
        p_element = container.find_all('p')[-1]  # Asumiendo que la fecha está en el último <p> del contenedor
        first_text = p_element.text.strip().split('\n')[0].strip() if p_element else ""
        readable_date = convert_duration_to_date(first_text)
        
        # Crear un diccionario para almacenar la data
        data = {
            "title": title,
            "duration": duration,
            "description": description,
            "fecha": readable_date
        }
        
        if description:
            data_list.append(data)
            print(data)

{'title': 'Javier Conte entrenador Majdalani Bosco', 'duration': '08:00', 'description': 'Con silvia Marucchi y Hernán', 'fecha': '8 de agosto de 2024 14:37:13'}
{'title': 'PNT YPF 05-08', 'duration': '00:23', 'description': 'PNT YPF 05-08', 'fecha': '5 de agosto de 2024 15:37:13'}
{'title': 'Columna de Sergio Wischñevsky', 'duration': '07:47', 'description': 'La habitual columna de #SiempreEsHoy', 'fecha': '8 de agosto de 2024 15:37:13'}
{'title': 'Gervasio Muñoz, Presidente de Inquilinos Agrupados', 'duration': '06:26', 'description': 'Escuchá la nota en Duro de Callar', 'fecha': '8 de agosto de 2024 11:37:13'}
{'title': 'La denuncia del hermano de Macri', 'duration': '11:36', 'description': 'Blindar el sentido del viento. La facho fest.Otro parteaguas: los que ven una dictadura civico-militar', 'fecha': '8 de agosto de 2024 15:37:13'}
{'title': 'Matías Cadaveira, Psicólogo (MN 40967)', 'duration': '11:34', 'description': 'Inclusión y cine: cómo las funciones distendidas ofrecen una 

##### Obtengo la descripción con más caracteres como para saber

In [8]:
def find_max_description_length(data_list):
    max_length = 0
    dictDescLenMax = {}
    for item in data_list:
        description = item.get("description", "")
        current_length = len(description)
        currentMax = item
        if current_length > max_length:
            max_length = current_length
            dictDescLenMax = currentMax
    return dictDescLenMax

dictDescLenMax = find_max_description_length(data_list).get("description")
max_length = len(dictDescLenMax)
print(f"La descripción con longitud máxima tiene: {max_length} caracteres con espacios")
print(f"y es: {dictDescLenMax}")


La descripción con longitud máxima tiene: 877 caracteres con espacios
y es: Este año será la 11°Edición del #MañanaEsMejor, el homenaje solidario a Spinetta que venimos haciendo con el apoyo de la cultura independiente y @conduciendoaconcienciaok.❤️Cada edición que celebramos es muy importante para nosotres. Promover la educación vial, concientizar cómo lo hacía el Flaco y además celebrarlos es un tesoro, uno de.los más importantes ... 💎Por eso este 8 de Agosto a partir de las 19 hs nos encontramos en @guajira.bar con este lineup del carajo:VanzaAgus DettbarnLa Selva de MiguelHelenNat Soule/ Nahuel PiscitelliGuido NicolásKaren Z en La cuadraMili MontoneSofia UzalJuan Francisco AltamirandaSerigrafía en Vivo : Rosal de Aquí / Diego BlockMusicaliza : El Tío Valen📚La entrada es una donación de 2 cuadernos de tapa dura. Todo lo recaudado será donado como siempre a las ESCUELAS RURALES del norte del país apadrinadas por @conduciendoaconcienciaok


### C - Pasos para enviar los recortes a la IA

#### 1ero: convertir el resultado del scrapping en un txt

In [9]:
def flatDictToTxt(dict_list, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for dictionary in dict_list:
            for key, value in dictionary.items():
                # Escribe cada llave y valor en el archivo
                file.write(f"{key}: {value} ")

file_path = 'output.txt'

# Escritura del archivo
flatDictToTxt(data_list, file_path)

#### 2do: cargar en la base de datos de vectores

###### Para probar: https://github.com/pgvector/pgvector

###### Para convertir el diccionario en un pdf o convertir un pdf a txt: https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

###### Para usar FAISS Instalación de miniconda3 y: https://docs.conda.io/projects/conda/en/stable/user-guide/getting-started.html

In [10]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_mistralai import MistralAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

mistral = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=api_key
)

# Specify the correct encoding
loader = TextLoader("output.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
texts = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(texts, mistral)

c:\Python311\Lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


### D - Consulta a la IA

###### Fuente: https://colab.research.google.com/github/mistralai/cookbook/blob/main/mistral/rag/basic_RAG.ipynb#scrollTo=d67a4729-cd2f-47e7-a4f6-f84a5677414f

#### Temas tratados en el día

In [11]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Define una interfaz retriever (ahí está el context)
retriever = vectorstore.as_retriever()
# Instancia el Chat
model = ChatMistralAI(api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Responde la siguiente pregunta basándote únicamente en el contexto provisto:

<context>
{context}
</context>

Question: {input}""")

# Crea una cadena de retrieval para responder a la pregunta
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "¿De qué temas se habló?"})
print(response["answer"])

Based on the provided context, the following topics were discussed:

1. Javier Conte's training session with Majdalani Bosco.
2. PNT (Noticias) from YPF on August 5, 2024.
3. Sergio Wischñevsky's regular column on #SiempreEsHoy.
4. An interview with Gervasio Muñoz, President of Inquilinos Agrupados, on Duro de Callar.
5. The denunciation made by the brother of Macri, comparing the current situation to a civico-military dictatorship.
6. An interview with Matías Cadaveira, a psychologist, discussing inclusive cinema and the new distended film showings adapted for neurodivergent individuals.
7. An interview with Lucas Gonzàlez, actor and creator of the show "Lucas Gonzàlez canta a Nacha Guevara".
8. An interview with esgrimist Nicolas Bermudez.
9. The special Lenguas originarias (Indigenous Languages) Encyclopedia Quichua.
10. The 11th edition of the #MañanaEsMejor event, a tribute to Spinetta.
11. A conversation with Gabriela Parigi about the play "Consagrada".
12. The Magnesium's role i

#### Pregunta de contexto

In [12]:
document_chain2 = create_stuff_documents_chain(model, prompt)
retrieval_chain2 = create_retrieval_chain(retriever, document_chain2)
response2 = retrieval_chain2.invoke({"input": "¿Qué porcentaje de la población rosarina es pobre?"})
print(response2["answer"])

There is no information about the poverty rate of the Rosarina population in the provided context.


#### Recorte de mayor duración

In [13]:
document_chain3 = create_stuff_documents_chain(model, prompt)
retrieval_chain3 = create_retrieval_chain(retriever, document_chain3)
response3 = retrieval_chain3.invoke({"input": "¿Cuál es el título de mayor duración?"})
print(response3["answer"])

El título de mayor duración es "42 Edición de Estación Central, la propuesta radial de la CTA Capital" con una duración de 111:02.
